In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, confusion_matrix
from sklearn.preprocessing import label_binarize

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
# from OpenMax import openmax



import time

In [2]:
head = {
            "model" : '',
            "attack_model": '',
            'epsilon': '',
            'Accuracy': '',
            'Macro Precision': '',
            'Weighted Precision': '',
            'Macro Recall': '',
            'Weighted Recall': '',
            'Macro F1': '',
            'Weighted F1': '',

        }
head = pd.DataFrame([head])
head.to_csv("./RGAN.csv", mode='a', index=False)


In [3]:
# def calculate_performance_metrics(X_test, y_test, model, model_name, attack_name, eps):
#     model.eval()
#     device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#     model.to(device)
    
#     all_preds = []
#     all_labels = []
#     probabilities = []

#     num_classes = len(np.unique(y_test))
    
#     X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
#     y_test_tensor = torch.tensor(y_test, dtype=torch.long)
#     test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
#     test_loader = DataLoader(dataset=test_dataset)

#     with torch.no_grad():
        
#         for inputs, labels in test_loader:
#             inputs, labels = inputs.to(device), labels.to(device)
#             outputs = model(inputs)
#             preds = torch.argmax(outputs, dim=1)
#             all_preds.extend(preds.cpu().numpy())
#             all_labels.extend(labels.cpu().numpy())
#             probabilities.extend(torch.nn.functional.softmax(outputs, dim=1).cpu().numpy())
        
#         all_preds = np.array(all_preds)
#         all_labels = np.array(all_labels)
#         probabilities = np.array(probabilities)

#         np.save(f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Defense_Label/UNSW_Def11/y_pred_{attack_name}{eps}_Def11.npy", all_preds)

        

In [4]:
def calculate_performance_metrics(X_test, y_test, model, model_name, attack_name, eps):
    model.eval()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    all_preds = []
    all_labels = []
    probabilities = []

    num_classes = len(np.unique(y_test))
    
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
    test_loader = DataLoader(dataset=test_dataset)

    with torch.no_grad():
        
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            probabilities.extend(torch.nn.functional.softmax(outputs, dim=1).cpu().numpy())
        
        all_preds = np.array(all_preds)
        all_labels = np.array(all_labels)
        probabilities = np.array(probabilities)
        
        accuracy = accuracy_score(all_labels, all_preds)

        precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(all_labels, all_preds, average='macro')
        precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')
    

        print(f"Accuracy: {accuracy}")
        
        print("\nmacro")
        print(f"Precision: {precision_macro}\nRecall: {recall_macro}\nF1 Score: {f1_macro}")
    
        print("\nweighted")
        print(f"Precision: {precision_weighted}\nRecall: {recall_weighted}\nF1 Score: {f1_weighted}")
        print()
        


        new_row = {
            "model" : model_name,
            "attack_model" : attack_name,
            'epsilon': eps,
            'Accuracy': accuracy,
            'Macro Precision': precision_macro,
            'Weighted Precision': precision_weighted,
            'Macro Recall': recall_macro,
            'Weighted Recall': recall_weighted,
            'Macro F1': f1_macro,
            'Weighted F1': f1_weighted,

        }
        new_row_df = pd.DataFrame([new_row])
        new_row_df.to_csv("./RGAN.csv", mode='a', index=False, header=False)



In [5]:
x_test = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
x_train = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_train.npy')
x_val = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_val.npy')
y_test = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')
y_train = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_train.npy')
y_val = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_val.npy')

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

Using cuda device


In [7]:
x_train_tensor = torch.tensor(x_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)

x_val_tensor = torch.tensor(x_val, dtype=torch.float32).to(device)
y_val_tensor = torch.tensor(y_val, dtype=torch.long).to(device)

train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True)

val_dataset = TensorDataset(x_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=100, shuffle=True)

In [8]:
# class DNNModel(nn.Module):
#     def __init__(self, input_size, output_size):
#         super(DNNModel, self).__init__()
#         self.fc1 = nn.Linear(input_size, 50)
#         self.bn1 = nn.BatchNorm1d(50)
#         self.fc2 = nn.Linear(50, 30)
#         self.bn2 = nn.BatchNorm1d(30)
#         self.fc3 = nn.Linear(30, 20)
#         self.bn3 = nn.BatchNorm1d(20)
#         self.fc4 = nn.Linear(20, output_size)

#     def forward(self, x):
#         x = self.bn1(F.relu(self.fc1(x)))
#         x = self.bn2(F.relu(self.fc2(x)))
#         x = self.bn3(F.relu(self.fc3(x)))
#         logits = self.fc4(x)
#         return logits

class DNNModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(DNNModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, 30)
        self.fc3 = nn.Linear(30, 20)
        self.fc4 = nn.Linear(20, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

class Generator(nn.Module):
    def __init__(self, input_size):
        super(Generator, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, input_size)

    def forward(self, x):
        perturbation = torch.tanh(self.fc1(x))
        perturbation = self.fc2(perturbation)
        perturbation = torch.clamp(perturbation, -0.1, 0.1)
        return perturbation

class Discriminator(nn.Module):
    def __init__(self, input_size):
        super(Discriminator, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x

In [9]:
input_size = x_train.shape[1]
output_size = len(np.unique(y_train))

In [10]:
def train_rgan(model, G, D, train_loader, val_loader, device, epochs=15, lr=0.001, patience=3, min_delta=0.001):
    optimizer_G = optim.Adam(G.parameters(), lr=lr)
    optimizer_D = optim.Adam(D.parameters(), lr=lr)
    optimizer_M = optim.Adam(model.parameters(), lr=lr)
    loss_function = nn.CrossEntropyLoss()

    best_loss = float("inf")
    patience_counter = 0

    for epoch in range(epochs):
        model.train()
        G.train()
        D.train()

        train_loss = 0.0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            adv_inputs = inputs + G(inputs).detach().to(device)
            adv_inputs = torch.clamp(adv_inputs, -1, 1)

            optimizer_D.zero_grad()
            real_loss = -torch.mean(D(inputs))
            fake_loss = torch.mean(D(G(inputs)))
            loss_D = real_loss + fake_loss
            loss_D.backward()
            optimizer_D.step()

            optimizer_G.zero_grad()
            fake_loss = -torch.mean(D(G(inputs)))
            fake_loss.backward()
            optimizer_G.step()

            optimizer_M.zero_grad()
            outputs = model(adv_inputs)
            loss_M = loss_function(outputs, labels)
            loss_M.backward()
            optimizer_M.step()

            train_loss += loss_M.item()

        avg_train_loss = train_loss / len(train_loader)

        model.eval()
        val_loss = 0.0
        correct_predictions = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = loss_function(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                correct_predictions += (predicted == labels).sum().item()

        avg_val_loss = val_loss / len(val_loader)
        val_accuracy = correct_predictions / len(val_loader.dataset)

        print(f"Epoch {epoch+1}, Training Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

        if best_loss - avg_val_loss > min_delta:
            best_loss = avg_val_loss
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= patience:
            print("Early stopping triggered")
            break


In [11]:
model = DNNModel(input_size, output_size).to(device)
G = Generator(input_size).to(device)
D = Discriminator(input_size).to(device)



In [12]:
# train_rgan(model, G, D, train_loader, val_loader, device)


In [13]:
# torch.save(model.state_dict(), "./RGAN.pt")

In [14]:
model.load_state_dict(torch.load("/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Defense/RGAN/RGAN.pt"))

/tmp/ipykernel_3676/2756394764.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Defense/RG

<All keys matched successfully>

In [15]:
# calculate_performance_metrics(x_test, y_test, model, 'DNN', 'baseline', '0')

In [16]:
def print_empty_file():
    new_row = {
        "model" : "0",
        "attack_model" : "0",
        'epsilon': "0",
        'Accuracy': "0",
        'Macro Precision': "0",
        'Weighted Precision': "0",
        'Macro Recall': "0",
        'Weighted Recall': "0",
        'Macro F1': "0",
        'Weighted F1': "0",
    }
    new_row_df = pd.DataFrame([new_row])
    new_row_df.to_csv("./RGAN.csv", mode='a', index=False, header=False)

In [17]:
# percentage = ["100", "50", "20", "1"]
# model_name = ["XGB", "RF", "DT" ]

percentage = ["10"]
model_name = ["RF"]

In [18]:
epsilon = 0
Def = "Def11"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]



base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

for m_name in model_name:
    for p in percentage:
        for attack in attack_names:
            print(f"start {m_name} {p} {attack}")
            
            x_path = f"{base_path}/{m_name}/UNSW_Input{p}/x_test_adv_{attack}_{Def}.npy"
            y_path = f"{base_path}/{m_name}/UNSW_Input{p}/y_test_adv_{attack}_{Def}.npy"

            try:
                x_test_adv = np.load(x_path)
                y_test_adv = np.load(y_path)
                print(x_test_adv.shape, y_test_adv.shape)

                m_per_name = f"{m_name}{p}"
                calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
            except FileNotFoundError:
                print(x_path, "not found")
                print_empty_file()


start RF 10 baseline
(26, 56) (26,)
Accuracy: 0.7307692307692307

macro
Precision: 0.66875
Recall: 0.7571428571428571
F1 Score: 0.6720720720720721

weighted
Precision: 0.8341346153846154
Recall: 0.7307692307692307
F1 Score: 0.7574497574497575

start RF 10 BIM
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF/UNSW_Input10/x_test_adv_BIM_Def11.npy not found
start RF 10 FGSM
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF/UNSW_Input10/x_test_adv_FGSM_Def11.npy not found
start RF 10 PGD
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF/UNSW_Input10/x_test_adv_PGD_Def11.npy not found
start RF 10 DF
(68, 56) (68,)
Accuracy: 0.04411764705882353

macro
Precision: 0.025
Recall: 0.13636363636363635
F1 Score: 0.04225352112676056

weighted
Precision: 0.008088235294117648
Recall: 0.04411764705882353
F1 Score: 0.013670256835128416

start RF 10 AutoPGD
(63, 56) (63,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.7142857142857143

macro
Precision: 0.5
Recall: 0.35714285714285715
F1 Score: 0.4166666666666667

weighted
Precision: 1.0
Recall: 0.7142857142857143
F1 Score: 0.8333333333333334

start RF 10 ZOO
(78, 56) (78,)
Accuracy: 0.7307692307692307

macro
Precision: 0.66875
Recall: 0.7571428571428571
F1 Score: 0.6720720720720721

weighted
Precision: 0.8341346153846154
Recall: 0.7307692307692307
F1 Score: 0.7574497574497574

start RF 10 CaFA
(243, 56) (243,)
Accuracy: 0.2674897119341564

macro
Precision: 0.5029852076278738
Recall: 0.5281512605042017
F1 Score: 0.22483870967741937

weighted
Precision: 0.9640814302886819
Recall: 0.2674897119341564
F1 Score: 0.3991840346918006

start RF 10 SINIFGSM
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF/UNSW_Input10/x_test_adv_SINIFGSM_Def11.npy not found
start RF 10 VNIFGSM
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF/UNSW_Input10/x_test_adv_VNIFGSM_Def11.npy not found


In [19]:
epsilon = 0
Def = "Def11"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]



base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

for m_name in model_name:
    for p in percentage:
        for attack in attack_names:
            print(f"start {m_name} {p} {attack}")
            
            x_path = f"{base_path}/{m_name}_ExcludeCaFA/UNSW_Input{p}/x_test_adv_{attack}_{Def}.npy"
            y_path = f"{base_path}/{m_name}_ExcludeCaFA/UNSW_Input{p}/y_test_adv_{attack}_{Def}.npy"

            try:
                x_test_adv = np.load(x_path)
                y_test_adv = np.load(y_path)
                print(x_test_adv.shape, y_test_adv.shape)

                m_per_name = f"{m_name}{p}_ExcludeCaFA"
                calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
            except FileNotFoundError:
                print(x_path, "not found")
                print_empty_file()


start RF 10 baseline
(38, 56) (38,)
Accuracy: 0.7105263157894737

macro
Precision: 0.4090909090909091
Recall: 0.421875
F1 Score: 0.4153846153846154

weighted
Precision: 0.6889952153110048
Recall: 0.7105263157894737
F1 Score: 0.6995951417004049

start RF 10 BIM
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_ExcludeCaFA/UNSW_Input10/x_test_adv_BIM_Def11.npy not found
start RF 10 FGSM
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_ExcludeCaFA/UNSW_Input10/x_test_adv_FGSM_Def11.npy not found
start RF 10 PGD
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_ExcludeCaFA/UNSW_Input10/x_test_adv_PGD_Def11.npy not found
start RF 10 DF
(14, 56) (14,)
Accuracy: 0.07142857142857142

macro
Precision: 0.08333333333333333
Recall: 0.05555555555555555
F1 Score: 0.06666666666666667

weighted
Precision: 0.10714285714285714
Recall: 0.07142857142857142
F1 Score: 0.08571428571428572

start RF 10 AutoPGD
(96, 56) (96,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.84375

macro
Precision: 0.5
Recall: 0.421875
F1 Score: 0.4576271186440678

weighted
Precision: 1.0
Recall: 0.84375
F1 Score: 0.9152542372881355

start RF 10 ZOO
(111, 56) (111,)
Accuracy: 0.7297297297297297

macro
Precision: 0.421875
Recall: 0.421875
F1 Score: 0.421875

weighted
Precision: 0.7297297297297297
Recall: 0.7297297297297297
F1 Score: 0.7297297297297297

start RF 10 CaFA
(10, 56) (10,)
Accuracy: 0.0

macro
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

weighted
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

start RF 10 SINIFGSM
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_ExcludeCaFA/UNSW_Input10/x_test_adv_SINIFGSM_Def11.npy not found
start RF 10 VNIFGSM
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_ExcludeCaFA/UNSW_Input10/x_test_adv_VNIFGSM_Def11.npy not found


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_

In [20]:
epsilon = 0
Def = "Def11"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]



base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

total_num_attack = ["2", "3"]
for num_attack in total_num_attack:
    for m_name in model_name:
        for p in percentage:
            for attack in attack_names:
                print(f"start {m_name} {p} {attack}")
                
                x_path = f"{base_path}/{m_name}_Exclude{num_attack}Attack/UNSW_Input{p}/x_test_adv_{attack}_{Def}.npy"
                y_path = f"{base_path}/{m_name}_Exclude{num_attack}Attack/UNSW_Input{p}/y_test_adv_{attack}_{Def}.npy"
    
                try:
                    x_test_adv = np.load(x_path)
                    y_test_adv = np.load(y_path)
                    print(x_test_adv.shape, y_test_adv.shape)
    
                    m_per_name = f"{m_name}{p}_Exclude{num_attack}Attack"
                    calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
                except FileNotFoundError:
                    print(x_path, "not found")
                    print_empty_file()


start RF 10 baseline
(32, 56) (32,)
Accuracy: 0.71875

macro
Precision: 0.7078431372549019
Recall: 0.740909090909091
F1 Score: 0.7046153846153846

weighted
Precision: 0.7732843137254901
Recall: 0.71875
F1 Score: 0.7288461538461539

start RF 10 BIM
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_Exclude2Attack/UNSW_Input10/x_test_adv_BIM_Def11.npy not found
start RF 10 FGSM
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_Exclude2Attack/UNSW_Input10/x_test_adv_FGSM_Def11.npy not found
start RF 10 PGD
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_Exclude2Attack/UNSW_Input10/x_test_adv_PGD_Def11.npy not found
start RF 10 DF
(37, 56) (37,)
Accuracy: 0.40540540540540543

macro
Precision: 0.25
Recall: 0.3409090909090909
F1 Score: 0.28846153846153844

weighted
Precision: 0.2972972972972973
Recall: 0.4054054054054054
F1 Score: 0.343035343035343

start RF 10 AutoPGD
(66, 56) (66,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.6818181818181818

macro
Precision: 0.5
Recall: 0.3409090909090909
F1 Score: 0.40540540540540543

weighted
Precision: 1.0
Recall: 0.6818181818181818
F1 Score: 0.8108108108108109

start RF 10 ZOO
(96, 56) (96,)
Accuracy: 0.71875

macro
Precision: 0.7078431372549019
Recall: 0.740909090909091
F1 Score: 0.7046153846153846

weighted
Precision: 0.7732843137254902
Recall: 0.71875
F1 Score: 0.7288461538461539

start RF 10 CaFA
(9, 56) (9,)
Accuracy: 0.8888888888888888

macro
Precision: 0.5
Recall: 0.4444444444444444
F1 Score: 0.47058823529411764

weighted
Precision: 1.0
Recall: 0.8888888888888888
F1 Score: 0.9411764705882353

start RF 10 SINIFGSM
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_Exclude2Attack/UNSW_Input10/x_test_adv_SINIFGSM_Def11.npy not found
start RF 10 VNIFGSM
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_Exclude2Attack/UNSW_Input10/x_test_adv_VNIFGSM_Def11.npy not found
start RF 10 baseline
(18, 56) (18,)
Accur

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.p

Accuracy: 0.0

macro
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

weighted
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

start RF 10 PGD
(13, 56) (13,)
Accuracy: 0.07692307692307693

macro
Precision: 0.5
Recall: 0.038461538461538464
F1 Score: 0.07142857142857142

weighted
Precision: 1.0
Recall: 0.07692307692307693
F1 Score: 0.14285714285714285

start RF 10 DF
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_Exclude3Attack/UNSW_Input10/x_test_adv_DF_Def11.npy not found
start RF 10 AutoPGD
(45, 56) (45,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.p

Accuracy: 0.6

macro
Precision: 0.5
Recall: 0.3
F1 Score: 0.375

weighted
Precision: 1.0
Recall: 0.6
F1 Score: 0.75

start RF 10 ZOO
(54, 56) (54,)
Accuracy: 0.6666666666666666

macro
Precision: 0.6666666666666666
Recall: 0.8
F1 Score: 0.625

weighted
Precision: 0.8888888888888888
Recall: 0.6666666666666666
F1 Score: 0.7083333333333334

start RF 10 CaFA
(3, 56) (3,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start RF 10 SINIFGSM
(1, 56) (1,)
Accuracy: 0.0

macro
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

weighted
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

start RF 10 VNIFGSM
(33, 56) (33,)
Accuracy: 0.030303030303030304

macro
Precision: 0.5
Recall: 0.015151515151515152
F1 Score: 0.029411764705882353

weighted
Precision: 1.0
Recall: 0.030303030303030304
F1 Score: 0.058823529411764705



/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_

In [17]:
epsilon = 0

Def = "Def11"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

rec_list = ["Euclidean", "cosine", "manhattan"]
for rec in rec_list:
    for attack in attack_names:
        print(f"start {attack}")
        
        x_path = f"{base_path}/Recommendation_{rec}/x_test_adv_{attack}_{Def}.npy"
        y_path = f"{base_path}/Recommendation_{rec}/y_test_adv_{attack}_{Def}.npy"
    
        try:
            x_test_adv = np.load(x_path)
            y_test_adv = np.load(y_path)
            m_per_name = f"Recommendation_{rec}"
            calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
        except FileNotFoundError:
            print(x_path, "not found")
            print_empty_file()



start baseline
Accuracy: 0.7647058823529411

macro
Precision: 0.6364535266974292
Recall: 0.8004354136429608
F1 Score: 0.645531914893617

weighted
Precision: 0.901508395052154
Recall: 0.7647058823529411
F1 Score: 0.8061576971214017

start BIM
Accuracy: 0.6640746500777605

macro
Precision: 0.5758720159827125
Recall: 0.7160291386115626
F1 Score: 0.5389212014978886

weighted
Precision: 0.8952263808096101
Recall: 0.6640746500777605
F1 Score: 0.735057203003658

start FGSM
Accuracy: 0.5641025641025641

macro
Precision: 0.5355481727574751
Recall: 0.5515911282545805
F1 Score: 0.5096153846153846

weighted
Precision: 0.6925973251554647
Recall: 0.5641025641025641
F1 Score: 0.6018244575936885

start PGD
Accuracy: 0.6640746500777605

macro
Precision: 0.5758720159827125
Recall: 0.7160291386115626
F1 Score: 0.5389212014978886

weighted
Precision: 0.8952263808096101
Recall: 0.6640746500777605
F1 Score: 0.735057203003658

start DF
Accuracy: 0.17857142857142858

macro
Precision: 0.20348837209302326
Recal

In [21]:
epsilon = 0
Def = "Def11"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]



base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

for m_name in model_name:
    for p in percentage:
        for attack in attack_names:
            print(f"start {m_name} {p} {attack}")
            
            x_path = f"{base_path}/{m_name}_Cluster/UNSW_Input{p}/x_test_adv_{attack}_{Def}.npy"
            y_path = f"{base_path}/{m_name}_Cluster/UNSW_Input{p}/y_test_adv_{attack}_{Def}.npy"

            try:
                x_test_adv = np.load(x_path)
                y_test_adv = np.load(y_path)
                print(x_test_adv.shape, y_test_adv.shape)

                m_per_name = f"{m_name}{p}_Cluster"
                calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
            except FileNotFoundError:
                print(x_path, "not found")
                print_empty_file()


start RF 10 baseline
(126, 56) (126,)
Accuracy: 0.7142857142857143

macro
Precision: 0.47303921568627455
Recall: 0.4625
F1 Score: 0.4650943396226415

weighted
Precision: 0.7637721755368814
Recall: 0.7142857142857143
F1 Score: 0.7374663072776281

start RF 10 BIM
(2, 56) (2,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start RF 10 FGSM
(2, 56) (2,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start RF 10 PGD
(2, 56) (2,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start RF 10 DF
(21, 56) (21,)
Accuracy: 0.3333333333333333

macro
Precision: 0.35
Recall: 0.19444444444444445
F1 Score: 0.25

weighted
Precision: 0.6
Recall: 0.3333333333333333
F1 Score: 0.42857142857142855

start RF 10 AutoPGD
(61, 56) (61,)
Accuracy: 0.14754098360655737

macro
Precision: 0.22083333333333333
Recall: 0.177

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [22]:
epsilon = 0
Def = "Def11"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]



base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

for m_name in model_name:
    for p in percentage:
        for attack in attack_names:
            print(f"start {m_name} {p} {attack}")
            
            x_path = f"{base_path}/{m_name}_ActiveLearning/UNSW_Input{p}/x_test_adv_{attack}_{Def}.npy"
            y_path = f"{base_path}/{m_name}_ActiveLearning/UNSW_Input{p}/y_test_adv_{attack}_{Def}.npy"

            try:
                x_test_adv = np.load(x_path)
                y_test_adv = np.load(y_path)
                print(x_test_adv.shape, y_test_adv.shape)

                m_per_name = f"{m_name}{p}_ActiveLearning"
                calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
            except FileNotFoundError:
                print(x_path, "not found")
                print_empty_file()


start RF 10 baseline
(25, 56) (25,)
Accuracy: 0.76

macro
Precision: 0.475
Recall: 0.3958333333333333
F1 Score: 0.4318181818181818

weighted
Precision: 0.9119999999999999
Recall: 0.76
F1 Score: 0.8290909090909091

start RF 10 BIM
(1, 56) (1,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start RF 10 FGSM
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_ActiveLearning/UNSW_Input10/x_test_adv_FGSM_Def11.npy not found
start RF 10 PGD
(1, 56) (1,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start RF 10 DF
(37, 56) (37,)
Accuracy: 0.13513513513513514

macro
Precision: 0.1
Recall: 0.14705882352941177
F1 Score: 0.11904761904761904

weighted
Precision: 0.0918918918918919
Recall: 0.13513513513513514
F1 Score: 0.1093951093951094

start RF 10 AutoPGD
(72, 56) (72,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.7916666666666666

macro
Precision: 0.5
Recall: 0.3958333333333333
F1 Score: 0.4418604651162791

weighted
Precision: 1.0
Recall: 0.7916666666666666
F1 Score: 0.8837209302325582

start RF 10 ZOO
(75, 56) (75,)
Accuracy: 0.76

macro
Precision: 0.475
Recall: 0.3958333333333333
F1 Score: 0.4318181818181818

weighted
Precision: 0.9119999999999999
Recall: 0.76
F1 Score: 0.8290909090909091

start RF 10 CaFA
(20, 56) (20,)
Accuracy: 0.6

macro
Precision: 0.5
Recall: 0.3
F1 Score: 0.375

weighted
Precision: 1.0
Recall: 0.6
F1 Score: 0.75

start RF 10 SINIFGSM
(2, 56) (2,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start RF 10 VNIFGSM
(1, 56) (1,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0



/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [23]:
epsilon = 0

Def = "Def11"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]


active_learning_name = ["DensityWeighted", "BatchMode"]

base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

for ac_name in active_learning_name:
    for m_name in model_name:
        for p in percentage:
            for attack in attack_names:
                print(f"start {m_name} {p} {attack}")
                
                x_path = f"{base_path}/{m_name}_ActiveLearning_{ac_name}/UNSW_Input{p}/x_test_adv_{attack}_{Def}.npy"
                y_path = f"{base_path}/{m_name}_ActiveLearning_{ac_name}/UNSW_Input{p}/y_test_adv_{attack}_{Def}.npy"
    
                try:
                    x_test_adv = np.load(x_path)
                    y_test_adv = np.load(y_path)
                    m_per_name = f"{m_name}{p}_ActiveLearning_{ac_name}"
                    calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
                except FileNotFoundError:
                    print(x_path, "not found")
                    print_empty_file()



start RF 10 baseline
Accuracy: 0.6199744735162731

macro
Precision: 0.6106276676568313
Recall: 0.6946243660318561
F1 Score: 0.5692993955573586

weighted
Precision: 0.8245656684396107
Recall: 0.6199744735162731
F1 Score: 0.6664069868088598

start RF 10 BIM
Accuracy: 0.5500566679259539

macro
Precision: 0.5589255033487999
Recall: 0.5797550141337999
F1 Score: 0.5228562542141623

weighted
Precision: 0.6909834373922757
Recall: 0.5500566679259539
F1 Score: 0.5811735969032091

start RF 10 FGSM
Accuracy: 0.5526421944332393

macro
Precision: 0.5850627379756055
Recall: 0.627524908481323
F1 Score: 0.5257806984270892

weighted
Precision: 0.7607650844392908
Recall: 0.5526421944332393
F1 Score: 0.5916596555811554

start RF 10 PGD
Accuracy: 0.5500566679259539

macro
Precision: 0.5589255033487999
Recall: 0.5797550141337999
F1 Score: 0.5228562542141623

weighted
Precision: 0.6909834373922757
Recall: 0.5500566679259539
F1 Score: 0.5811735969032091

start RF 10 DF
Accuracy: 0.4713493530499076

macro
Prec

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classifi

Accuracy: 0.1875

macro
Precision: 0.11904761904761904
Recall: 0.234375
F1 Score: 0.15789473684210525

weighted
Precision: 0.09523809523809523
Recall: 0.1875
F1 Score: 0.12631578947368421

start RF 10 AutoPGD
Accuracy: 0.8

macro
Precision: 0.4918032786885246
Recall: 0.40540540540540543
F1 Score: 0.4444444444444444

weighted
Precision: 0.9704918032786886
Recall: 0.8
F1 Score: 0.877037037037037

start RF 10 ZOO
Accuracy: 0.8022813688212928

macro
Precision: 0.6853900709219858
Recall: 0.7992216117216118
F1 Score: 0.7088230284448986

weighted
Precision: 0.8767607798721787
Recall: 0.8022813688212928
F1 Score: 0.8248619074357236

start RF 10 CaFA
Accuracy: 0.47719298245614034

macro
Precision: 0.47188735898178874
Recall: 0.3614130073655942
F1 Score: 0.34268613375887946

weighted
Precision: 0.8707737783716771
Recall: 0.47719298245614034
F1 Score: 0.6081686251016832

start RF 10 SINIFGSM
Accuracy: 0.0

macro
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

weighted
Precision: 0.0
Recall: 0.0
F1 Scor

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_